## Loading packages

In [ ]:
#Access the server
import jwt_access

url = "url of server here"
username = "your username"
password = "your password"

auth = jwt_access.Authenticate(url, username, password)

In [2]:
#Set path for accessing ecosystem python wrappers
import sys
sys.path.append("/path of ecosystem server python wrappers")

In [6]:
#Load packages
import pymongo
from bson.son import SON
import pprint
import pandas as p
import json
import numpy
import operator
import datetime
import time

from apis import functions as uf
from apis import data_management_engine as d
from apis import data_munging_engine as dm
from apis import worker_h2o as hw
from apis import prediction_engine as pe

## Set up mongo databse connection ##

In [7]:
client = pymongo.MongoClient(
   "mongodb://connection details")

In [8]:
db = client["database name"]

In [ ]:
db.list_collection_names()

## Manual feature store enrichment example ##

#### Get a list of field names in the feature stores

In [10]:
fields_pipeline = [
    {"$project":{"arrayofkeyvalue":{"$objectToArray":"$$ROOT"}}},
    {"$unwind":"$arrayofkeyvalue"},
    {"$group":{"_id":None,"allkeys":{"$addToSet":"$arrayofkeyvalue.k"}}}
  ]

In [6]:
cursor = db.featureStoreOfCategories.aggregate(fields_pipeline)
doc = cursor.next()
listOfFields = doc["allkeys"]
if "_id" in listOfFields: listOfFields.remove("_id")

#### List of the year month week combos used in the feature store names

In [295]:
list_of_months = ["list of month indicators to add to base feature stores"]

#### Add one step lags to all of the feature stores

In [ ]:
for j in range(len(list_of_months)-1):
    print(j)
    current_fs = "baseFeatureStore"+list_of_months[j+1]
    previous_fs = "baseFeatureStore"+list_of_months[j]
    write_fs = "baseFeatureStoreLags"+list_of_months[j]
    ratio_pipeline = [
                        {
                        "$lookup":{
                                "from":previous_fs
                                ,"localField":"_id"
                                ,"foreignField":"_id"
                                ,"as":"subs"
                                }
                        }
                        ,{"$unwind":"$subs"}
                        ,{
                        "$addFields":{
                                    }
                        }
                        ,{"$unset":"subs"}
                        ,{"$out":write_fs}
                    ]
    
    for i in listOfFields:
        add_field = i+"Ratio"
        add_field_appear = i+"Appear"
        current_value = "$"+ i
        previous_value = "$subs." + i
        ratio_pipeline[2]["$addFields"][add_field]={"$switch":{"branches":[
                         {"case":{"$and":[{"$ne":[{"$type":current_value}, "missing"]},{"$ne":[{"$type":previous_value}, "missing"]}]}, "then":{"$divide":[current_value,previous_value]}}
                        ,{"case":{"$and":[{"$ne":[{"$type":current_value}, "missing"]},{"$eq":[{"$type":previous_value}, "missing"]}]}, "then":0}
                        ], "default":None}}
        ratio_pipeline[2]["$addFields"][add_field_appear]={"$cond":[{"$and":[{"$eq":[{"$type":current_value}, "missing"]},{"$ne":[{"$type":previous_value}, "missing"]}]},1,0]}
    
    db[current_fs].aggregate(ratio_pipeline)

#### Add behaviour change indiciator showing number of categories appearing or disappearing

In [ ]:
for j in range(len(list_of_months)-1):
    add_dict = {"$addFields":
                       {
                        "appear":{"$add":[]}
                        ,"disappear":{"$add":[]}
                       }
           }
    for i in listOfFields:
        field_disappear = "$"+i+"Ratio"
        field_appear = "$"+i+"Appear"
        add_dict["$addFields"]["appear"]["$add"].append({"$cond":[{"$eq":[field_appear,1]},1,0]})
        add_dict["$addFields"]["disappear"]["$add"].append({"$cond":[{"$eq":[field_disappear,0]},1,0]})

    write_fs = "fnbPrismFeatureStoreLags"+list_of_months[j]
    print(write_fs)
    behav_change_pipeline = [
        add_dict
        ,{"$out":write_fs}
    ]

    db[write_fs].aggregate(behav_change_pipeline)


## Build an h2o model using the ecosystem.ai packages

In [ ]:
#Set column data types - this is approximate and should be checked
fs = "training feature store"
export_store = fs
export_store_file = export_store+".csv"
example_data = d.get_data(auth,"fnb_stage1", fs, "{}", 10, "{}", 0)
example_data_frame = p.DataFrame(example_data)
listOfColumnNames = list(example_data_frame.columns)
export_projection = ""
for i in listOfColumnNames:
    export_projection = export_projection + i + ","

export_projection = export_projection[:-1]
d.export_documents(auth,export_store, "csv", "fnb_stage1", export_store, "{}", "{}", export_projection, 75000)
parsed = hw.file_to_frame(auth,export_store_file, 1, "comma")

In [ ]:
pathvar = "/Users/ecosystem/data/fnb_2020_05/" + fs +".csv"
hexframename = uf.save_userframe(example_data_frame, fs, pathvar)

In [ ]:
#Set model training parameters
model_id = fs+modelIdInc
model_purpose = "Prediction of whether nonbehavioural prism model is correct"
model_params = { 
        "predict_id": fs,
        "description": descrip,
        "model_id": model_id,
        "model_type": "AUTOML",
        "frame_name": hexframename,
        "frame_name_desc": descrip,
        "model_purpose": model_purpose,
        "version": modelIdInc,
        "model_parms": {
              "algo": "H2O-AUTOML",
              "training_frame": hexframename,
              "validation_frame": hexframename,
              "max_models": 10,
              "stopping_tolerance": 0.005,
              "note_stop": "stopping_tolerance of 0.001 for 1m rows and 0.004 for 100k rows",
              "max_runtime_secs": 3600,
              "stopping_rounds": 15,
              "stopping_metric": "AUTO",
              "nfolds": 4,
              "note_folds": "nfolds=0 will disable the stacked ensemble creation process",
              "response_column": "prismResponse",
              "ignored_columns": [            
                  "prismResponse",
                  "other columns in feature store you don"t want to be included in the model"
              ],
              "hidden": [
                "1"
              ],
              "exclude_algos": [
                "GLM",
                "StackedEnsemble",
                "XGBoost",
                "DeepLearning",
                "GBM",
                "Any algorithms that you don"t want to be included in the automl run"
              ]
            }
    }
#Start the model training
hw.train_model(auth, model_id, "automl", json.dumps(model_params["model_parms"]))

In [ ]:
#Save the model
h2o_name = "name of the best model"
correct_models_dict[j] = h2o_name
zip_name = h2o_name + ".zip"
hw.download_model_mojo(auth,h2o_name)
high_level_mojo=hw.get_train_model(auth, h2o_name, "eric")
model_to_save = high_level_mojo["models"][0]
model_to_save["model_identity"] = h2o_name
model_to_save["userid"] = "jayvanzyl"
model_to_save["timestamp"] = "time_stamp"
pe.save_model(auth,model_to_save)

#See some statistics from the saved model
pe.get_user_model(auth,h2o_name)
stats = hw.get_model_stats(auth,h2o_name,"ecosystem","variable_importances")